# Notebook E-tivity 1 CE4021

Student name: Aurelie Montfrond

Student ID:

22289003  

## Imports

In [1]:
#None

If you believe required imports are missing, please contact your moderator.

<hr style="border:2px solid gray"> </hr>

## Task

You may add as many cells as you require to complete this task. Refer to the Rubric for E-tivity 1 to see how you will be assessed.

In [ ]:
# Option 1: Symbolic calculation of the derivative
#symbolically calculate the derivative of a polynomial with one variable.

#example 1 = 3x + 4 - 5
#example 2 = 5x - 3x + 2
#example 3 = 2x - x + 2 + 4x

polynomial1 = [3, 4, 5]
polynomial2 = [5, 3, 2]
polynomial3 = [2, 1, 2, 4]

#fonction to find the derivative of a polynomial . The input and output are polynomials as a list. 
# I name the fonction 'derivative' and pass one polynomial variable as its argument . 
# I create an array/list named 'res' to store the result so the derivative .
# I read the polynomial which is a list with a for loop until the end of the list using the 'i' as index.
# I append the calculation of the derivative to the 'res' list.
# I print the result of the calculation by returning the variable 'res' 

def derivative(polynomial1):
        res=[]
        for i in range(len(polynomial1)-1):
            res.append((len(polynomial1)-i-1)*polynomial1[i])
        return res


#evaluate a polynomial with a given value for its variable (x in this case) 

x = 3 

def evaluate(x):
   
     return 5 * x - 3 * x + 2

 

<hr style="border:2px solid gray"> </hr>

## Reflection

Write your reflection in below cell. With reference to the Rubric for E-tivity 1:
- Provide an accurate description of your code with advantages and disadvantages of design choices.
- Compare your approach to alternative (peer) approaches.
- Clearly describe how you have used your peers' work/input and how this has affected your own understanding / insights.

If you have not used peer input, you may state this, but your submission history in Gitlab should clearly show this is the case.

In [ ]:
I explained the code with comments and I guess the advantages are that the code is consice and clear i feel.
It is simple and the disadvantages I feel the fact that I was not sure at least without spending more time to research
how and if i'ts possibe . I think it's possible to pass several variables in the argument of the function. So I could have
passed the 3 examples variables such as def derivative(polynomial1, polynomial2, polynomial3): but then i would have to do 3 for loops 
and i think we can only return once a function. I guess I think I would need to create 3 separate functions to calculate the 
derivative of each example. 

For the evaluate function , I think it's simple but maybe I could have used a for loop . I think it depends 
if for the purpose of the task we just define the variable, I guess it's ok to calculate the way I did it but with user
input we could use a for loop.

